In [1]:
#!/bin/env python


import sys,os,os.path,time
sys.path.append(os.path.expanduser('/global/u1/r/rgupta2/.local/lib/python3.7/site-packages/'))

!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pims
!{sys.executable} -m pip install --user netCDF4
!{sys.executable} -m pip install --user xarray
!{sys.executable} -m pip install --user tqdm
!{sys.executable} -m pip install --user kornia

# export PYTHONPATH="${PYTHONPATH}:/usr/local/lib/python2.7/site-packages:/usr/lib/python2.7/site-packages"
import numpy as np
import time
import torch, kornia
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob
import tensorflow as tf
tf.enable_eager_execution()


# from mpl_toolkits.basemap import Basemap


file_path = "/project/projectdirs/dasrepo/mustafa/datasets/Rayleigh_Benard/result_rb_2d__Ra_2.5e8__Pr_0.71__maxMach_0.1__t_D_max_diffusive_scaling__0.4.nc"



  Using cached https://files.pythonhosted.org/packages/00/08/bbf9f5465c92ea6577bc5824fea21d2583e24e94fe10cc853587ffcacda0/PIMS-0.4.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/75/ae/fe46f5371105508a209fe6162e7e7b11db531a79d2eabcd24566b8b1f534/slicerator-1.0.0-py3-none-any.whl
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/common/software/tensorflow/intel-tensorflow/1.13.1-py36/lib/python3.6/site-packages/slicerator-1.0.0.dist-info'
Consider using the `--user` option or check the permissions.



In [2]:
t = time.time()

ds = xr.open_dataset(file_path, decode_times=False	)
print("Time takes : ", time.time()-t)

Time takes :  163.82600903511047


In [ ]:
"""
This block plots the differnet velocity field vectors for considered time range and how normalizing them affects them

"""




st_time = 10000
end_time =15000
image_size = 256

number_of_timesteps = 500

ux_data_for_tke = ds['u_x'][st_time : end_time].values
uy_data_for_tke = ds['u_y'][st_time : end_time].values


save_dir_path = "/global/cscratch1/sd/rgupta2/backup/StyleGAN/dataset/"


ux_data_mean_tke = np.mean(ux_data_for_tke, axis = 0)
uy_data_mean_tke = np.mean(uy_data_for_tke, axis = 0)

print(ux_data_mean_tke.shape)


# load_path = os.path.join(dataset_location + "tke_average_energies.npy")
# my_dict_back = np.load(load_path, allow_pickle=True)
# ux_average_over_time = my_dict_back.item()["{}_ux".format(image_size)]
# uy_average_over_time = my_dict_back.item()["{}_uy".format(image_size)]

# print(ux_data_for_tke.shape)
# print(uy_data_for_tke.shape)

res_list = [8, 16, 32, 64, 128, 256]
tt = time.time()
Data = {}
for res in res_list:
    print(ux_data_mean_tke.shape)
    print("Time taken to complete iteration", time.time() - tt)
    
    
    ux_data = ux_data_mean_tke
    
    strides_x =  [x * ux_data.itemsize for x in [ux_data.shape[-1] * ux_data.shape[-2], res, ux_data.shape[-1], 1]]
    u_x = np.lib.stride_tricks.as_strided(ux_data, (number_of_timesteps, ux_data.shape[-1]//image_size, image_size, image_size), strides_x)

    ux_data_mean_tke = np.mean(u_x, axis=0)
    
    print(" strides : {} u_x.shape {} ux_data_mean_tke : {} ".format(strides, u_x.shape, ux_data_mean_tke.shape))
    
    
    
    uy_data = uy_data_mean_tke

    strides_y =  [x * uy_data.itemsize for x in [uy_data.shape[-1] * uy_data.shape[-2], image_size, uy_data.shape[-1], 1]]
    u_y = np.lib.stride_tricks.as_strided(uy_data, (number_of_timesteps, ux_data.shape[-1]//image_size, image_size, image_size), strides_y)
    uy_data_mean_tke = np.mean(u_y, axis=0)
    
    print("u_y.shape {}, uy_data_mean_tke {} ".format(u_y.shape, uy_data_mean_tke.shape))
    
    
    
    Data["{}_ux".format(res)] = ux_data_mean_tke
    Data["{}_uy".format(res)] = uy_data_mean_tke
    
    
    
     
save_path = os.path.join(save_dir_path , "rbc_{}/std_normalized/".format(number_of_timesteps))

if not os.path.exists(save_path):
    os.makedirs(save_path)

np.save( str(save_path) + "tke_average_energies_{}_{}.npy".format(st_time, end_time), Data)
    
print("Time taken to complete iteration", time.time() - tt)


In [2]:
"""
"""

from glob import glob

load_dir_path = "/global/cscratch1/sd/rgupta2/backup/StyleGAN/dataset/rbc_500/max/"

np_files =  sorted(glob("{}/*.npy".format(load_dir_path)))[:]

tmp_ux = []
tmp_uy = []

for file in np_files:
    Data = np.load(file)
    print(Data.shape)
    tmp_ux.append(Data[:,0,:,:])
    tmp_uy.append(Data[:,1,:,:])
    

ux_data_for_tke = np.concatenate(tmp_ux, axis=0)
uy_data_for_tke = np.concatenate(tmp_uy, axis=0)

ux_data_mean_tke = np.mean(ux_data_for_tke, axis = 0)
uy_data_mean_tke = np.mean(uy_data_for_tke, axis = 0)

print(ux_data_mean_tke.shape)
res = ux_data_mean_tke.shape[-1]


stored_data = {}

stored_data["{}_ux".format(res)] = ux_data_mean_tke
stored_data["{}_uy".format(res)] = uy_data_mean_tke

np.save( str(load_dir_path) + "tke_average_energies_{}.npy".format(res), stored_data)
    
print("Time taken to complete iteration", time.time() - tt)


# load_path = os.path.join(dataset_location + "tke_average_energies.npy")
# my_dict_back = np.load(load_path, allow_pickle=True)
# ux_average_over_time = my_dict_back.item()["{}_ux".format(image_size)]
# uy_average_over_time = my_dict_back.item()["{}_uy".format(image_size)]

# print(ux_data_for_tke.shape)
# print(uy_data_for_tke.shape)

# res_list = [8, 16, 32, 64, 128, 256]
# tt = time.time()
# Data = {}
# for res in res_list:
#     print(ux_data_mean_tke.shape)
#     print("Time taken to complete iteration", time.time() - tt)
    
    
#     ux_data = ux_data_mean_tke
    
#     strides_x =  [x * ux_data.itemsize for x in [ux_data.shape[-1] * ux_data.shape[-2], res, ux_data.shape[-1], 1]]
#     u_x = np.lib.stride_tricks.as_strided(ux_data, (number_of_timesteps, ux_data.shape[-1]//image_size, image_size, image_size), strides_x)

#     ux_data_mean_tke = np.mean(u_x, axis=0)
    
#     print(" strides : {} u_x.shape {} ux_data_mean_tke : {} ".format(strides, u_x.shape, ux_data_mean_tke.shape))
    
    
    
#     uy_data = uy_data_mean_tke

#     strides_y =  [x * uy_data.itemsize for x in [uy_data.shape[-1] * uy_data.shape[-2], image_size, uy_data.shape[-1], 1]]
#     u_y = np.lib.stride_tricks.as_strided(uy_data, (number_of_timesteps, ux_data.shape[-1]//image_size, image_size, image_size), strides_y)
#     uy_data_mean_tke = np.mean(u_y, axis=0)
    
#     print("u_y.shape {}, uy_data_mean_tke {} ".format(u_y.shape, uy_data_mean_tke.shape))
    
    
    
#     Data["{}_ux".format(res)] = ux_data_mean_tke
#     Data["{}_uy".format(res)] = uy_data_mean_tke
    
    
    
     



(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(3500, 2, 256, 256)
(256, 256)


NameError: name 'tt' is not defined